<h1>Getting started with Archicad Python API</h1><a class="anchor" id="chapter1"></a>
<h2>Introduction</h2>
<p>You can start from here: <a href="https://pypi.org/project/archicad/">Archicad Python Package Webpage</a></p>
<br>
Here you can find the package, the minimum information regarding installation and some details regarding the package.
<br><br>
Under the Documentation section you can find some useful links:
<br>
<ul>
<li><a href="https://archicadapi.graphisoft.com/archicadPythonPackage/archicad.html">Reference Manual</a>
<li><a href="https://archicadapi.graphisoft.com/JSONInterfaceDocumentation/#Introduction">Documentation of Archicad's new JSON interface</a>
<li><a href="https://graphisoft.com/downloads/python">Official Graphisoft Homepage / downloads / python</a></li>
</ul>
<b>What is the purpose of this repo if Graphisoft has the User Manual, and those links detailing the API?</b>
<br>
<p>The truth to be told I have found those docs to be a bit confusing and not well detailed. I think if you are just getting started with python and APIs you migh be confused too.<br>
With this repo I try to help you to understand how the Archicad python (JSON) API works, what the main concepts are.<br>
I hope if you read through this repo you will get the hang of it or at least get the general sense of how this API works and you can make your own scripts.</p> 

<h2>Preparation</h2>
<h4>Get the project file</h4>
<b>Note:</b>To get the required file you need to have a Graphisoft account, if you don't have yet it is very easy to create and it is free too.
<br>
<dd><a href="https://graphisoft.com/dl/Python/Room%20report%20generator.zip">Room report generator.zip</a></dd>
<br>
<h4>Getting ready for scripting</h4>
<b>Note:</b> We are not using the built in Archicad python addon to run our codes instead, we are utilizing an IDE to create and run our python scripts (i.e. VS Code, Jupyter Notebook etc.)<br><br>


<h2>Scripting</h2>
Some things we always need to do to get the environment we are using up and running:
<br><br>
<ol>
<li>We need to open our Archicad software and open the  project we would like to manipulate.</li>
<li>we need to import the Archicad connection module and establish the connection to the Archicad program with the project open.</li>



In [1]:
from archicad import ACConnection

conn = ACConnection.connect()
assert conn

<h4>The Archicad API has three main pillars we can use to manipulate the project</h4>
<ol>
<li>Commands</li>
<dt>These are the collection of the Archicad JSON interface commands, used to give instruction (requests) to the Archicad Software itself(e.g. get certain elements' id, set a certain value to an element etc.)</dt><br>
<li>Types</li>
<dt>Types used to manipulate different objects (e.g.element Id, element value etc.)</dt><br>
<li>Utilities</li>
<dt>Independent utilities doing things with the elements, properties etc.</dt>
</ol>

In [17]:
acc = conn.commands
act = conn.types
acu = conn.utilities

<h4>Example of a command</h4>
We can check if the connection is alive with the IsAlive() command:<br>
<br>
<dd><b>IsAlive()</b> → bool<br><br>
Checks if the Archicad connection is alive.
<br><br>
Returns:<br>
<i>A list of elements.</i>
<br>
Return type:<br>
<i>Returns true if the connection is alive</i>
<br>
Return type:<br>
<i>bool
</dd>
</dl>

In [18]:
print(acc.IsAlive())

True


<h2>Getting elements from the connected plan.</h2>
<br>To get elements from the connected plan we practically get the 'guid' of the element.<br><br>
- The 'guid' is a <i>Globally Unique Identifier</i> also referred to as 'UUID' - <i>Universally Unique Identifier</i> -
<br><br><h4>Some options to get the 'guid' of the elements:</h4>
<dl>
<dt><li>Using the GetElementsByType() command (1 step process):</li></dt><br>
<dd><b>GetElementsByType(</b><i>elementType: str</i><b>)</b> → List[ElementIdArrayItem]<br><br>
Returns the identifier of every element of the given type on the plan.<br>
<br>Parameters:<br>
<i>elementType (str) – The type of an element.</i>
<br>
Returns:<br>
<i>A list of elements.</i>
<br>
Return type:<br>
<i>list of ElementIdArrayItem</i>
</dd>
</dl>
<br>
Supported element types:<br>
<a href="https://archicadapi.graphisoft.com/JSONInterfaceDocumentation/#GetElementsByType">https://archicadapi.graphisoft.com/JSONInterfaceDocumentation/#GetElementsByType</a>
<br>
<br>List of these elements as of Apr.2023:<br>
Wall Column Beam Window Door Object Lamp Slab Roof Mesh Zone CurtainWall Shell Skylight Morph Stair Railing Opening

In [22]:
elements = acc.GetElementsByType("Beam")
print(elements)
print(f"length of the elements' list: {len(elements)}, type: {type(elements)}")
print(f"The elementId of the first element in the list: {elements[0].elementId}")

[ElementIdArrayItem {'elementId': {'guid': 'BC551E01-954A-48CD-A742-B17F959A843C'}}, ElementIdArrayItem {'elementId': {'guid': '1DD49D40-562B-4CE1-BDE7-592478EBCC92'}}, ElementIdArrayItem {'elementId': {'guid': '25EF8392-D2AC-4816-AE2C-C25726E17E0C'}}, ElementIdArrayItem {'elementId': {'guid': '57CF6693-EE63-4FE0-8C93-67A22A11E0A1'}}]
length of the elements' list: 4, type: <class 'list'>
The elementId of the first element in the list: ElementId {'guid': 'BC551E01-954A-48CD-A742-B17F959A843C'}



<dt><li>Using the GetElementsByClassification() command (2 steps process):</li></dt><br>
<dd>To get element using the classification Id we need to know the classification Id of the element:<br><br>
We can find all the classifications in Archicad using the <i>Classification Manager</i> (<b>Options > Classification Manager</b>):<br>
<a href="url">https://helpcenter.graphisoft.com/user-guide/76741/</a><br><br>
<img src="Classific_man.jpg" alt="Classification Manager"><br><br>
To get the 'guid' of the classification Id we use the FindClassificationItemInSystem() utility (1st step):<br><br>
<b>FindClassificationItemInSystem(</b><i>system_name: str, item_id: str</i><b>)</b> → Optional[ClassificationItemInTree]<br><br>
Finds the classification item in a system.<br>
<br>Parameters:<br>
<i>system_name – Given in the 'Name' field under 'Classification System Definition' in the Classification Manager.<br>
item_id – To be find in the Classification Manager, e.g. in the classification tree under the search box.
</i><br>
Returns:<br>
<i>ClassificationItemInTree object which contains the ClassificationItemId, id, name, description of the ItemId and all the children ItemIds' classificationItem classes with the guid, id, name and description.</i>
<br>
Return type:<br>
<i>class of ClassificationItemInTree</i>
</dd>
</dl>

In [39]:
classificationItem = acu.FindClassificationItemInSystem(
    'ARCHICAD Classification', 'Window')
print(f'Type of the found item: {type(classificationItem)}')
print(classificationItem)
print(type(classificationItem.classificationItemId))
print(f'we can get the guid only if we need: {classificationItem.classificationItemId.guid}\n')

Type of the found item: <class 'archicad.releases.ac26.b3000types.ClassificationItemInTree'>
ClassificationItemInTree {'classificationItemId': {'guid': 'D8A32916-7247-451F-B6A2-0711D228146C'}, 'id': 'Window', 'name': '', 'description': '', 'children': [{'classificationItem': {'classificationItemId': {'guid': '1C49465B-361F-40F3-AA6C-64C94D74A871'}, 'id': 'Lightdome', 'name': '', 'description': ''}}, {'classificationItem': {'classificationItemId': {'guid': '7EE7257C-1A9E-49DF-AE79-2C09F2ADA3C7'}, 'id': 'Skylight', 'name': '', 'description': ''}}]}
<class 'archicad.releases.ac26.b3000types.ClassificationItemId'>
we can get the guid only if we need: d8a32916-7247-451f-b6a2-0711d228146c



<dd>Using the GetElementsByClassification() command with the classificationItemId got from step 1 (2nd step):<br><br>
<b>GetElementsByClassification(</b><i>classificationItemId: ClassificationItemId</i><b>)</b> → List[ElementIdArrayItem]<br><br>
Returns the identifier of every element with the given classification identifier.<br>
<br>Parameters:<br>
<i>classificationItemId (ClassificationItemId) – The identifier of a classification item.</i>
<br>
Returns:<br>
<i>A list of elements.</i>
<br>
Return type:<br>
<i>list of ElementIdArrayItem
</dd>
</dl>

In [72]:
elements = acc.GetElementsByClassification(
    classificationItem.classificationItemId)
print("Getting the elements' ids:")
print(*elements, sep='\n')
print(type(elements), f', length of elements: {len(elements)}')
print(f"First element Id: {elements[0].elementId}")

Getting the elements' ids:
ElementIdArrayItem {'elementId': {'guid': 'EAA67613-636E-4C66-B0DC-231A6DFD45F7'}}
ElementIdArrayItem {'elementId': {'guid': '17F46B70-DADE-4441-8AD1-173A04671C01'}}
ElementIdArrayItem {'elementId': {'guid': '69D6E4A1-3FD2-4283-816B-F9B75640E729'}}
ElementIdArrayItem {'elementId': {'guid': 'CD17D722-65E4-4264-8479-D59202FC6E6E'}}
ElementIdArrayItem {'elementId': {'guid': '8880C88E-7652-45DA-BD37-2604ED36729E'}}
ElementIdArrayItem {'elementId': {'guid': '7B32239F-A123-4D19-A412-91E0168883C3'}}
ElementIdArrayItem {'elementId': {'guid': 'A025C6AA-7088-41DB-A6DD-9B64DBF7C8B6'}}
ElementIdArrayItem {'elementId': {'guid': '219E6870-DCAB-4F71-A271-D6216B664044'}}
ElementIdArrayItem {'elementId': {'guid': 'D67FFF1D-DB78-4884-A520-755D8C89F118'}}
ElementIdArrayItem {'elementId': {'guid': '45298259-0D6F-4566-86C5-5713AC42B08E'}}
ElementIdArrayItem {'elementId': {'guid': 'C248FA74-18AD-4C97-BADE-0B4692BF6B4C'}}
<class 'list'> , length of elements: 11
First element Id: El

<dl>
<dt><li>Using the GetAllElements() command (1 step process):</li></dt><br>
<dd><b>GetAllElements()</b> → List[ElementIdArrayItem]<br><br>
Returns the identifier of every element in the current plan.<br>
<br>Parameters:<br>
<i>elementType (str) – The type of an element.</i>
<br>
Returns:<br>
<i>A list of elements.</i>
<br>
Return type:<br>
<i>list of ElementIdArrayItem</i>
</dd>
</dl>

In [43]:
all_elements_lst = acc.GetAllElements()
print(f'Number of all elements in the project (length of the all_elements_lst): {len(all_elements_lst)}')
print(*all_elements_lst[:5], sep='\n')

Number of all elements in the project (length of the all_elements_lst): 45
ElementIdArrayItem {'elementId': {'guid': 'A8138BA2-01DA-4193-AA14-974E9CA78483'}}
ElementIdArrayItem {'elementId': {'guid': '29ABB091-81C6-4828-B3BF-B7B268D0B45B'}}
ElementIdArrayItem {'elementId': {'guid': 'C054C783-FF9B-43D8-A9A0-3C779CF096BA'}}
ElementIdArrayItem {'elementId': {'guid': 'B885985B-85A7-4377-A477-4B88189822CC'}}
ElementIdArrayItem {'elementId': {'guid': '2F01B74E-3C9C-4FFD-BEC7-5ECAF4B106AD'}}


<h3 style='margin-bottom: -5px'>Important to consider:</h3><br>
With the method GetElementsByType() we are using the type so we are getting all the elements of the requested type.<br>
With the method GetElementsByClassification(), we are using the classifiction which can be inactive, not defined or used appropriately or whtever the case may be.In this case we have to check if those elements were selected we really wanted to.<br>
e.g. GetElementsByType() select the slabs but GetElementsByClassification() select the slabs only with the slab clasification!  


<h2> Element properties:</h2>
To obtain an <i>element property</i> we need to get the <i>'guid'</i> - <i>Globally Unique Identifier</i> (or 'UUID' - <i>Universally Unique Identifier</i>) of the required property.

In order to get the property 'guid' we first need to get the element property ID:
<dl>
<dt><li>method 1 using the GetBuiltInPropertyId() utility (1 step process):</li></dt><br>
<dd><b>GetBuiltInPropertyId(</b><i>name: str</i><b>)</b> → PropertyId<br><br>
Returns the PropertyId of the corresponding built-in property.</dd>
</dl>

In [44]:
propertyId = acu.GetBuiltInPropertyId('General_ElementID')
print(propertyId)
# we can make a property Id list:
propertyId_lst = [propertyId, acu.GetBuiltInPropertyId('General_Height')]
print(propertyId_lst)

PropertyId {'guid': '7E221F33-829B-4FBC-A670-E74DABCE6289'}
[PropertyId {'guid': '7E221F33-829B-4FBC-A670-E74DABCE6289'}, PropertyId {'guid': 'C4B62357-1289-4D43-A3F6-AB02B192864C'}]


<dl>
<dt><li>method 2 using the BuiltInPropertyUserId() type (2 steps process):</li></dt><br>
<dd>Getting the Built in property user id (1st step):
<br><br>
<b>BuiltInPropertyUserId(</b><i>nonLocalizedName: str, type: str = 'BuiltIn'</i><b>)</b><br><br>
Returns The unique identifier of a Built-In Property, identified by its name.
<br><br>
Variables:<br>
<i>nonLocalizedName (str) – The non-localized name of the Built-In Property.<br>
type (str) – EMPTY STRING</dd>
</dl>

In [45]:
builtin_p_u_id = act.BuiltInPropertyUserId("General_ElementID")
print(builtin_p_u_id)
# we can make a property Id list:
builtin_p_u_id_lst = [builtin_p_u_id, act.BuiltInPropertyUserId("General_Height")]
print(builtin_p_u_id_lst)

BuiltInPropertyUserId {'nonLocalizedName': 'General_ElementID', 'type': 'BuiltIn'}
[BuiltInPropertyUserId {'nonLocalizedName': 'General_ElementID', 'type': 'BuiltIn'}, BuiltInPropertyUserId {'nonLocalizedName': 'General_Height', 'type': 'BuiltIn'}]


<dd>Using the GetPropertyIds() command with the BuiltInPropertyUserId list got from step 1 (2nd step):<br><br>
<b>GetPropertyIds(</b><i>properties: List [PropertyUserId]</i><b>)</b> → List [PropertyIdOrError]<br><br>
Returns the identifiers of property definitions for the requested property names.<br>
<br>Parameters:<br>
<i>properties (list of PropertyUserId) – A list of PropertyUserId objects.</i>
<br>
Returns:<br>
<i>A list of property identifiers or errors.</i>
<br>
Return type:<br>
<i>list of PropertyIdOrError</i>
</dd>
</dl>

In [46]:
prop_ids = acc.GetPropertyIds(builtin_p_u_id_lst)
print(prop_ids)

[PropertyIdArrayItem {'propertyId': {'guid': '7E221F33-829B-4FBC-A670-E74DABCE6289'}}, PropertyIdArrayItem {'propertyId': {'guid': 'C4B62357-1289-4D43-A3F6-AB02B192864C'}}]


<h4>Available element properties:</h4>
Link to the properties:<br>
<a href="url">https://gdl.graphisoft.com/reference-guide/built-in-property-guide</a><br><br>
List of the available properties (Element-related built-in property IDs):<br>
<p style="font-size: em;">Builtin.General_3DLength (introduced in ARCHICAD 25)
<br>
Builtin.General_3DPerimeter (introduced in ARCHICAD 25)<br>
Builtin.General_Area (introduced in ARCHICAD 25)<br>
Builtin.General_BottomElevationToFirstReferenceLevel (introduced in ARCHICAD 25)<br>
Builtin.General_BottomElevationToHomeStory (introduced in ARCHICAD 25)<br>
Builtin.General_BottomElevationToProjectZero (introduced in ARCHICAD 25)<br>
Builtin.General_BottomElevationToSeaLevel (introduced in ARCHICAD 25)<br>
Builtin.General_BottomElevationToSecondReferenceLevel (introduced in ARCHICAD 25)<br>
Builtin.General_ConditionalBottomSurfaceArea (introduced in ARCHICAD 25)<br>
Builtin.General_ConditionalTopSurfaceArea (introduced in ARCHICAD 25)<br>
Builtin.General_ConditionalVolume (introduced in ARCHICAD 25)<br>
Builtin.General_CrossSectionAreaAtBeginCut (introduced in ARCHICAD 25)<br>
Builtin.General_CrossSectionAreaAtEndCut (introduced in ARCHICAD 25)<br>
Builtin.General_CrossSectionHeightAtBeginCut (introduced in ARCHICAD 25)<br>
Builtin.General_CrossSectionHeightAtBeginPerpendicular (introduced in ARCHICAD 25)<br>
Builtin.General_CrossSectionHeightAtEndCut (introduced in ARCHICAD 25)<br>
Builtin.General_CrossSectionHeightAtEndPerpendicular (introduced in ARCHICAD 25)<br>
Builtin.General_CrossSectionWidthAtBeginCut (introduced in ARCHICAD 25)<br>
Builtin.General_CrossSectionWidthAtBeginPerpendicular (introduced in ARCHICAD 25)<br>
Builtin.General_CrossSectionWidthAtEndCut (introduced in ARCHICAD 25)<br>
Builtin.General_CrossSectionWidthAtEndPerpendicular (introduced in ARCHICAD 25)<br>
Builtin.General_ElementID (introduced in ARCHICAD 25)<br>
Builtin.General_ElevationToFirstReferenceLevel (introduced in ARCHICAD 25)<br>
Builtin.General_ElevationToProjectZero (introduced in ARCHICAD 25)<br>
Builtin.General_ElevationToSeaLevel (introduced in ARCHICAD 25)<br>
Builtin.General_ElevationToSecondReferenceLevel (introduced in ARCHICAD 25)<br>
Builtin.General_ElevationToStory (introduced in ARCHICAD 25)<br>
Builtin.General_FloorPlanHolesPerimeter (introduced in ARCHICAD 25)<br>
Builtin.General_FloorPlanPerimeter (introduced in ARCHICAD 25)<br>
Builtin.General_FromZone (introduced in ARCHICAD 25)<br>
Builtin.General_FromZoneNumber (introduced in ARCHICAD 25)<br>
Builtin.General_GrossVolume (introduced in ARCHICAD 25)<br>
Builtin.General_Height (introduced in ARCHICAD 25)<br>
Builtin.General_Holes3DPerimeter (introduced in ARCHICAD 25)<br>
Builtin.General_HomeOffset (introduced in ARCHICAD 25)<br>
Builtin.General_HotlinkAndElementID (introduced in ARCHICAD 25)<br>
Builtin.General_HotlinkMasterID (introduced in ARCHICAD 25)<br>
Builtin.General_InsulationSkinThickness (introduced in ARCHICAD 25)<br>
Builtin.General_LastIssueID (introduced in ARCHICAD 25)<br>
Builtin.General_LastIssueName (introduced in ARCHICAD 25)<br>
Builtin.General_LibraryPartName (introduced in ARCHICAD 25)<br>
Builtin.General_Locked (introduced in ARCHICAD 25)<br>
Builtin.General_NetBottomSurfaceArea (introduced in ARCHICAD 25)<br>
Builtin.General_NetEdgeSurfaceArea (introduced in ARCHICAD 25)<br>
Builtin.General_NetTopSurfaceArea (introduced in ARCHICAD 25)<br>
Builtin.General_NetVolume (introduced in ARCHICAD 25)<br>
Builtin.General_OpeningNumber (introduced in ARCHICAD 25)<br>
Builtin.General_OwnerID (introduced in ARCHICAD 25)<br>
Builtin.General_RelatedZoneName (introduced in ARCHICAD 25)<br>
Builtin.General_RelatedZoneNumber (introduced in ARCHICAD 25)<br>
Builtin.General_SlantAngle (introduced in ARCHICAD 25)<br>
Builtin.General_SurfaceArea (introduced in ARCHICAD 25)<br>
Builtin.General_Thickness (introduced in ARCHICAD 25)<br>
Builtin.General_ToZone (introduced in ARCHICAD 25)<br>
Builtin.General_ToZoneNumber (introduced in ARCHICAD 25)<br>
Builtin.General_TopElevationToFirstReferenceLevel (introduced in ARCHICAD 25)<br>
Builtin.General_TopElevationToHomeStory (introduced in ARCHICAD 25)<br>
Builtin.General_TopElevationToProjectZero (introduced in ARCHICAD 25)<br>
Builtin.General_TopElevationToSeaLevel (introduced in ARCHICAD 25)<br>
Builtin.General_TopElevationToSecondReferenceLevel (introduced in ARCHICAD 25)<br>
Builtin.General_TopLinkStory (introduced in ARCHICAD 25)<br>
Builtin.General_TopOffset (introduced in ARCHICAD 25)<br>
Builtin.General_Type (introduced in ARCHICAD 25)<br>
Builtin.General_UniqueID (introduced in ARCHICAD 25)<br>
Builtin.General_Width (introduced in ARCHICAD 25)<br>
Builtin.Zone_AreaReducement (introduced in ARCHICAD 25)<br>
Builtin.Zone_CalculatedArea (introduced in ARCHICAD 25)<br>
Builtin.Zone_ExtractedColumnArea (introduced in ARCHICAD 25)<br>
Builtin.Zone_ExtractedCurtainWallArea (introduced in ARCHICAD 25)<br>
Builtin.Zone_ExtractedFillArea (introduced in ARCHICAD 25)<br>
Builtin.Zone_ExtractedLowArea (introduced in ARCHICAD 25)<br>
Builtin.Zone_ExtractedWallArea (introduced in ARCHICAD 25)<br>
Builtin.Zone_FloorThickness (introduced in ARCHICAD 25)<br>
Builtin.Zone_MeasuredArea (introduced in ARCHICAD 25)<br>
Builtin.Zone_NetArea (introduced in ARCHICAD 25)<br>
Builtin.Zone_NetPerimeter (introduced in ARCHICAD 25)<br>
Builtin.Zone_Perimeter (introduced in ARCHICAD 25)<br>
Builtin.Zone_ReducedArea (introduced in ARCHICAD 25)<br>
Builtin.Zone_TotalExtractedArea (introduced in ARCHICAD 25)<br>
Builtin.Zone_WallInsetBackSideSurfaceArea (introduced in ARCHICAD 25)<br>
Builtin.Zone_WallInsetSideSurfaceArea (introduced in ARCHICAD 25)<br>
Builtin.Zone_WallInsetTopSurfaceArea (introduced in ARCHICAD 25)<br>
Builtin.Zone_WallsPerimeter (introduced in ARCHICAD 25)<br>
Builtin.Zone_ZoneCategory (introduced in ARCHICAD 25)<br>
Builtin.Zone_ZoneCategoryCode (introduced in ARCHICAD 25)<br>
Builtin.Zone_ZoneName (introduced in ARCHICAD 25)<br>
Builtin.Zone_ZoneNumber (introduced in ARCHICAD 25)



<h4>Getting the details of the selected elements using property ids from method 1:</h4>

In [49]:
prop_ids_of_elements = acc.GetPropertyValuesOfElements(elements, propertyId_lst)
print('''---------------------------------------------
details of the list of PropertyValuesWrapper:
---------------------------------------------''')
print(prop_ids_of_elements)
print(type(prop_ids_of_elements))
print(len(prop_ids_of_elements))
print(prop_ids_of_elements[0])
print(type(prop_ids_of_elements[0]))
print(prop_ids_of_elements[0].propertyValues)
print(type(prop_ids_of_elements[0].propertyValues))
print(len(prop_ids_of_elements[0].propertyValues))
print(prop_ids_of_elements[0].propertyValues[0])
print(type(prop_ids_of_elements[0].propertyValues[0]))
print('''
-------------------------------------------
element 0, property 1. (General_ElementID):
-------------------------------------------''')
print(prop_ids_of_elements[0].propertyValues[0].propertyValue, type(prop_ids_of_elements[0].propertyValues[0].propertyValue))
print(prop_ids_of_elements[0].propertyValues[0].propertyValue.value, type(prop_ids_of_elements[0].propertyValues[0].propertyValue.value))
print(prop_ids_of_elements[0].propertyValues[0].propertyValue.type, type(prop_ids_of_elements[0].propertyValues[0].propertyValue.type))
print(prop_ids_of_elements[0].propertyValues[0].propertyValue.status, type(prop_ids_of_elements[0].propertyValues[0].propertyValue.status))
print('''
----------------------------------------
element 0, property 2. (General_Height):
----------------------------------------''')
print(prop_ids_of_elements[0].propertyValues[1].propertyValue, type(prop_ids_of_elements[0].propertyValues[1].propertyValue))
print(prop_ids_of_elements[0].propertyValues[1].propertyValue.value, type(prop_ids_of_elements[0].propertyValues[1].propertyValue.value))
print(prop_ids_of_elements[0].propertyValues[1].propertyValue.type, type(prop_ids_of_elements[0].propertyValues[1].propertyValue.type))
print(prop_ids_of_elements[0].propertyValues[1].propertyValue.status, type(prop_ids_of_elements[0].propertyValues[1].propertyValue.status))

---------------------------------------------
details of the list of PropertyValuesWrapper:
---------------------------------------------
[PropertyValuesWrapper {'propertyValues': [{'propertyValue': {'value': 'WD - 004', 'type': 'string', 'status': 'normal'}}, {'propertyValue': {'value': 1.5, 'type': 'length', 'status': 'normal'}}]}, PropertyValuesWrapper {'propertyValues': [{'propertyValue': {'value': 'WD - 004', 'type': 'string', 'status': 'normal'}}, {'propertyValue': {'value': 1.5, 'type': 'length', 'status': 'normal'}}]}, PropertyValuesWrapper {'propertyValues': [{'propertyValue': {'value': 'WD - 007', 'type': 'string', 'status': 'normal'}}, {'propertyValue': {'value': 1.5, 'type': 'length', 'status': 'normal'}}]}, PropertyValuesWrapper {'propertyValues': [{'propertyValue': {'value': 'WD - 008', 'type': 'string', 'status': 'normal'}}, {'propertyValue': {'value': 1.5, 'type': 'length', 'status': 'normal'}}]}, PropertyValuesWrapper {'propertyValues': [{'propertyValue': {'value': 'WD

<h4>Getting the details of the selected elements using property ids from method 2:</h4>

In [50]:
prop_ids_of_elements = acc.GetPropertyValuesOfElements(elements, prop_ids)
print('''---------------------------------------------
details of the list of PropertyValuesWrapper:
---------------------------------------------''')
print(prop_ids_of_elements)
print(type(prop_ids_of_elements))
print(len(prop_ids_of_elements))
print(prop_ids_of_elements[0])
print(type(prop_ids_of_elements[0]))
print(prop_ids_of_elements[0].propertyValues)
print(type(prop_ids_of_elements[0].propertyValues))
print(len(prop_ids_of_elements[0].propertyValues))
print(prop_ids_of_elements[0].propertyValues[0])
print(type(prop_ids_of_elements[0].propertyValues[0]))
print('''
-------------------------------------------
element 0, property 1. (General_ElementID):
-------------------------------------------''')
print(prop_ids_of_elements[0].propertyValues[0].propertyValue, type(prop_ids_of_elements[0].propertyValues[0].propertyValue))
print(prop_ids_of_elements[0].propertyValues[0].propertyValue.value, type(prop_ids_of_elements[0].propertyValues[0].propertyValue.value))
print(prop_ids_of_elements[0].propertyValues[0].propertyValue.type, type(prop_ids_of_elements[0].propertyValues[0].propertyValue.type))
print(prop_ids_of_elements[0].propertyValues[0].propertyValue.status, type(prop_ids_of_elements[0].propertyValues[0].propertyValue.status))
print('''
----------------------------------------
element 0, property 2. (General_Height):
----------------------------------------''')
print(prop_ids_of_elements[0].propertyValues[1].propertyValue, type(prop_ids_of_elements[0].propertyValues[1].propertyValue))
print(prop_ids_of_elements[0].propertyValues[1].propertyValue.value, type(prop_ids_of_elements[0].propertyValues[1].propertyValue.value))
print(prop_ids_of_elements[0].propertyValues[1].propertyValue.type, type(prop_ids_of_elements[0].propertyValues[1].propertyValue.type))
print(prop_ids_of_elements[0].propertyValues[1].propertyValue.status, type(prop_ids_of_elements[0].propertyValues[1].propertyValue.status))

---------------------------------------------
details of the list of PropertyValuesWrapper:
---------------------------------------------
[PropertyValuesWrapper {'propertyValues': [{'propertyValue': {'value': 'WD - 004', 'type': 'string', 'status': 'normal'}}, {'propertyValue': {'value': 1.5, 'type': 'length', 'status': 'normal'}}]}, PropertyValuesWrapper {'propertyValues': [{'propertyValue': {'value': 'WD - 004', 'type': 'string', 'status': 'normal'}}, {'propertyValue': {'value': 1.5, 'type': 'length', 'status': 'normal'}}]}, PropertyValuesWrapper {'propertyValues': [{'propertyValue': {'value': 'WD - 007', 'type': 'string', 'status': 'normal'}}, {'propertyValue': {'value': 1.5, 'type': 'length', 'status': 'normal'}}]}, PropertyValuesWrapper {'propertyValues': [{'propertyValue': {'value': 'WD - 008', 'type': 'string', 'status': 'normal'}}, {'propertyValue': {'value': 1.5, 'type': 'length', 'status': 'normal'}}]}, PropertyValuesWrapper {'propertyValues': [{'propertyValue': {'value': 'WD

<h2>Modify element properties' values:</h2>
<h4>Workflow</h4>
<dl>
    <dt><ol><li>Preparing Element property values</dt>
    <dd><b>ElementPropertyValue(</b><i>elementId: ElementId, propertyId: PropertyId, propertyValue: NormalOrUserUndefinedPropertyValue</i><b>)</b> utility:<br><br>
    A property value with the identifiers of the property and its owner element.<br><br>
Variables:<br>
<i>elementId (ElementId) – The identifier of an element.
<br>
propertyId (PropertyId) – The identifier of a property.
<br>
propertyValue (NormalOrUserUndefinedPropertyValue) – A normal or a userUndefined property value.</li>

In [83]:
# Preparing a new element property list with multiple list comprehension
# in order to modify the window IDs to 'Main window - Increasing index number from 1'
# and modify the window height to 800 mm for the first and increase this by +100 mm for the following and +100 mm for the next and so on.
print(f'Total number of elements to modify: {len(elements)}')
new_el_pr = [act.ElementPropertyValue(elements[i].elementId,
                                    propertyId_lst[l],
                                    [act.NormalStringPropertyValue, act.NormalLengthPropertyValue][l]([f'Main window - {i+1}', 0.8+0.1*i][l])) for i in range(len(elements)) for l in range(len(propertyId_lst))]
print(*new_el_pr, sep='\n') # using * and sep argument to print every element in new line
print(f'Total number of elements modified: {int(len(new_el_pr)/len(propertyId_lst))}')
print(f"""Total number of element's properties modified - length of propertyId_lst for each element,
in our case this is 2 for each element (General_elementID and General_Height): {int(len(new_el_pr))}""")

Total number of elements to modify: 11
ElementPropertyValue {'elementId': {'guid': 'EAA67613-636E-4C66-B0DC-231A6DFD45F7'}, 'propertyId': {'guid': '7E221F33-829B-4FBC-A670-E74DABCE6289'}, 'propertyValue': {'value': 'Main window - 1', 'type': 'string', 'status': 'normal'}}
ElementPropertyValue {'elementId': {'guid': 'EAA67613-636E-4C66-B0DC-231A6DFD45F7'}, 'propertyId': {'guid': 'C4B62357-1289-4D43-A3F6-AB02B192864C'}, 'propertyValue': {'value': 0.8, 'type': 'length', 'status': 'normal'}}
ElementPropertyValue {'elementId': {'guid': '17F46B70-DADE-4441-8AD1-173A04671C01'}, 'propertyId': {'guid': '7E221F33-829B-4FBC-A670-E74DABCE6289'}, 'propertyValue': {'value': 'Main window - 2', 'type': 'string', 'status': 'normal'}}
ElementPropertyValue {'elementId': {'guid': '17F46B70-DADE-4441-8AD1-173A04671C01'}, 'propertyId': {'guid': 'C4B62357-1289-4D43-A3F6-AB02B192864C'}, 'propertyValue': {'value': 0.9, 'type': 'length', 'status': 'normal'}}
ElementPropertyValue {'elementId': {'guid': '69D6E4A1

<dt><ol start=2><li>Setting Element property values to elements</dt>
    <dd><b>SetPropertyValuesOfElements(</b><i>elementPropertyValues: List[ElementPropertyValue]</i><b>)</b> command:<br><br>
    Sets the property values of elements.<br><br>
Parameters:<br>
<i>elementPropertyValues (list of ElementPropertyValue) – A list of element property values.</i>
<br>
Returns:
<br>
<i>A list of execution results.</i>
<br>
Return type:
<br>
<i>list of ExecutionResult</i>
</li>

In [84]:
acc.SetPropertyValuesOfElements(new_el_pr)

[SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'success': True},
 SuccessfulExecutionResult {'succe

<b>Note:</b> As you have probably noticed the changes doesn't follow any common logic e.g. clockwise.<br>
To pinpoint an element we can use for example the the <b>Get3DBoundingBoxes()</b> command and based on the elements' positions we can sort the element list as we like.

<h2>Modify element classification value:</h2>
<h4>Workflow</h4>
<dl>
    <dt><ol><li>Getting the classificationId value using the above already used</dt>
    <dd><b>FindClassificationItemInSystem(</b><i>system_name: str, item_id: str</i><b>)</b> utility.<br><br>

In [93]:
classificationItem = acu.FindClassificationItemInSystem(
    'ARCHICAD Classification', 'Door')
print(f'Type of the found item: {type(classificationItem)}')
print(classificationItem)
print(type(classificationItem.classificationItemId))
print(f'we can get the guid only if we need: {classificationItem.classificationItemId.guid}\n')

Type of the found item: <class 'archicad.releases.ac26.b3000types.ClassificationItemInTree'>
ClassificationItemInTree {'classificationItemId': {'guid': '0747DE76-67A9-4FD3-8321-8D5F938271FA'}, 'id': 'Door', 'name': '', 'description': '', 'children': [{'classificationItem': {'classificationItemId': {'guid': '9377292D-0D12-4347-A63C-DBE4487B3D52'}, 'id': 'Gate', 'name': '', 'description': ''}}, {'classificationItem': {'classificationItemId': {'guid': 'A331CC36-C023-4817-BB98-8C3F07DD9661'}, 'id': 'Trapdoor', 'name': '', 'description': ''}}]}
<class 'archicad.releases.ac26.b3000types.ClassificationItemId'>
we can get the guid only if we need: 0747de76-67a9-4fd3-8321-8d5f938271fa



<dt><ol start=2><li>To prepare the element classification value besides the classification Id we need to get the the <i>Classification System Ids</i></dt>
    <dd><b>GetClassificationSystemIds(</b><i></i><b>)</b>  → List[ClassificationSystemIdArrayItem] command:<br><br>
    Returns the list of available classification systems.<br><br>
Returns:
<br>
<i>A list of classification system identifiers.</i>
<br>
Return type:
<br>
<i>list of ClassificationSystemIdArrayItem</i>
</li>

In [94]:
acc.GetClassificationSystemIds()[0].classificationSystemId

ClassificationSystemId {'guid': '632C30E5-2B82-4A75-BFAD-D52FEBCE9A4A'}

<dt><ol start=3><li>Using Classification Id type prepare the new classification Id</dt>
    <dd><b>ClassificationId(</b><i>classificationSystemId: ClassificationSystemId, classificationItemId: Optional[ClassificationItemId] = None</i><b>)</b> type:<br><br>
    The element classification identifier.<br><br>
Variables:
<br><i>
classificationSystemId (ClassificationSystemId) – The identifier of a classification system.
classificationItemId (ClassificationItemId, optional) – The identifier of a classification item.
<br>
</li>

In [95]:
cl_id = act.ClassificationId(acc.GetClassificationSystemIds()[0].classificationSystemId, classificationItem.classificationItemId)
print(cl_id)

ClassificationId {'classificationSystemId': {'guid': '632C30E5-2B82-4A75-BFAD-D52FEBCE9A4A'}, 'classificationItemId': {'guid': '0747DE76-67A9-4FD3-8321-8D5F938271FA'}}


<dt><ol start=4><li>Using ElementClassification type prepare the new element classicifaction</dt>
    <dd><b>ElementClassification(</b><i>elementId: ElementId, classificationId: ClassificationId</i><b>)</b> type:<br><br>
The classification of an element.<br><br>
Variables:
<br>
<i>elementId (ElementId) – The identifier of an element.
classificationId (ClassificationId) – The element classification identifier.
<br>
</li>

In [96]:
el_cl = act.ElementClassification(elements[0].elementId, cl_id)
print(el_cl)

ElementClassification {'elementId': {'guid': 'EAA67613-636E-4C66-B0DC-231A6DFD45F7'}, 'classificationId': {'classificationSystemId': {'guid': '632C30E5-2B82-4A75-BFAD-D52FEBCE9A4A'}, 'classificationItemId': {'guid': '0747DE76-67A9-4FD3-8321-8D5F938271FA'}}}


<dt><ol start=5><li>Using SetClassificationsOfElements command to set the element new classification</dt>
    <dd><b>SetClassificationsOfElements(</b><i>elementClassifications: List[ElementClassification]</i><b>)</b> command:<br><br>
Sets the classifications of elements. In order to set the classification of an element to unclassified, omit the classificationItemId field.<br><br>
Parameters:
<br>
<i>elementClassifications (list of ElementClassification) – A list of element classification identifiers.</i>
<br>
Returns:
<br>
<i>A list of execution results.</i>
<br>
Return type:
<br>
<i>list of ExecutionResult</i>
</li>

In [97]:
acc.SetClassificationsOfElements([el_cl])

[SuccessfulExecutionResult {'success': True}]

With the above method we modified the classification of the first window element in the element list to 'Door'.<br>
If you check in Archicad the 800mm wide window element (down left corner of the building) you can see it is a window however the classification is changed to 'Door'. 

I hope this quick introduction will be a good base to your Archicad python API journey.